In [275]:
from collections import defaultdict, Counter
import string
import re
from itertools import groupby
from operator import itemgetter
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer 
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

%matplotlib inline

In [276]:
# open docs file and read its lines
with open("train.dat", "r") as fh:
    lines = fh.readlines()

In [277]:
# transform docs into lists of words
raw_lines = [l.split() for l in lines]
labels = list(map(lambda x: int(x[0]), raw_lines))
docs = list(map(lambda x: x[1:], raw_lines))

In [132]:
# Add validation set
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size=0.01)
docs = x_train

In [278]:
# Definition of preprocess functions

# Functional functions
def compose2(f, g):
    return lambda *a, **kw: f(g(*a, **kw))

def compose (*functions):
    def inner(arg):
        for f in reversed(functions):
            arg = f(arg)
        return arg
    return inner

# Filtering
has_valid_len = lambda w: len(w) >= 4
is_nonstop_word = lambda w: w not in ENGLISH_STOP_WORDS
is_valid_word = lambda w: has_valid_len(w) and is_nonstop_word(w)

# Mapping
to_lower = lambda w: w.lower()
remove_punc = lambda w: w.translate(str.maketrans('', '', string.punctuation))
remove_num = lambda w: re.sub(r'\d+', '', w)

map_word = compose(to_lower, remove_punc, remove_num)

# This mapping is expensive, so do it after filtering
ps = PorterStemmer() 
stem = lambda w: ps.stem(w)

def preprocess(docs):
    docs = [ [map_word(t) for t in d ] for d in docs ]
    docs = [ [t for t in d if is_valid_word(t)] for d in docs ]
#     docs = [ [stem(t) for t in d ] for d in docs ]
    return docs

def preprocess2(docs):
    docs = preprocess(docs)
    return [ [stem(t) for t in d ] for d in docs ]

In [279]:
docs_train = preprocess2(docs)

In [280]:
docs_test = preprocess2(x_test)

In [281]:
# Create sparse matrices, which requires aggregration of term IDs for both training and test data.

def calc_term_ids(docs):
    r""" The docs should be the combination of training set and test set."""
    term_ids = {}
    curr_term_id = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in term_ids:
                term_ids[w] = curr_term_id
                curr_term_id += 1
    return (term_ids, nnz)

def build_sparse_matrix(docs, term_ids = {}, nnz = 0):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    ncols = len(term_ids)
    assert(ncols != 0)

    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows + 1, dtype=np.int)
    row_id = 0  # document ID / row counter
    acc = 0  # non-zero counter

    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k, _ in cnt.most_common())
        curr_doc_len = len(keys)
        for i, key in enumerate(keys):
            ind[acc + i] = term_ids[key]
            val[acc + i] = cnt[key]
        ptr[row_id + 1] = ptr[row_id] + curr_doc_len
        acc += curr_doc_len
        row_id += 1

    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

In [282]:
# Build shared termID dictionary
term_ids, nnz = calc_term_ids([*docs_train, *docs_test])

In [283]:
TRAIN_SIZE = 10000
SAMPLE_SIZE = 100

In [284]:
# mat_train = build_sparse_matrix(docs_train, term_ids, nnz)
mat_train = build_sparse_matrix(docs_train[:TRAIN_SIZE], term_ids, nnz)

In [285]:
# mat_test = build_sparse_matrix(docs_test, term_ids, nnz)
mat_test = build_sparse_matrix(docs_test[:SAMPLE_SIZE], term_ids, nnz)

In [286]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat
    
def csr_normalize(mat, copy=False, **kargs):
    r""" csr_idf + csr_l2normalize"""
    if copy is True:
        mat = mat.copy()
    csr_idf(mat)
    csr_l2normalize(mat)
    if copy is True:
        return mat

In [287]:
mat_test = csr_normalize(mat_test, copy=True)
mat_train = csr_normalize(mat_train, copy=True)

In [288]:
K = 2
train_data = mat_train

def predict(test_data, verbose=False):
    nrows = test_data.shape[0]
    predictions = []
    for i, data in enumerate(test_data):
        if verbose:
            print( "- Starting test data %s..." % (i) )
        predictions.append(knn(data, train_data, k=K, verbose=verbose))
        
    return predictions

# Cosine similarity
from scipy.sparse.linalg import norm
def dist(a, b):
    dp = a.dot(b.T).todense().item()
    return dp / ( norm(a) * norm(b))

def knn(point, cluster, k, verbose=False):
    dists = []
    
    # Calculate distance b/ the given node to all other nodes
    for i, node in enumerate(cluster):
        d = dist(point, node)
        if verbose:
            print( "- Compared with train data %s. Dist: %s" % (i, d) )
        dists.append({"dist": d, "label": y_train[i]})
    
    # Sort and truncate by K
    dists = sorted(dists, reverse=True, key = lambda i: i["dist"])[:k]
    if verbose:
        print( "- Dist truncated by K=%d:" % (k), dists )

    # Add weight for classification
    weights = {}
    for label, v in groupby(dists, key = lambda i: i["label"]):
        weights[label] = sum(item["dist"] for item in list(v))
    
    return max(weights.items(), key=itemgetter(1))[0]
    

In [290]:
p = predict(mat_test, verbose=False)

In [291]:
print(p)
print(y_test[:SAMPLE_SIZE])

[5, 3, 4, 5, 1, 4, 4, 3, 5, 1, 1, 3, 4, 4, 5, 4, 3, 3, 5, 5, 5, 1, 5, 3, 5, 5, 5, 4, 4, 4, 4, 5, 1, 5, 4, 4, 3, 1, 4, 3, 3, 5, 1, 5, 5, 5, 5, 1, 1, 5, 5, 4, 1, 4, 5, 1, 5, 2, 5, 1, 2, 3, 2, 4, 3, 5, 1, 1, 4, 1, 5, 1, 4, 2, 5, 5, 5, 1, 5, 5, 5, 1, 3, 2, 5, 3, 1, 5, 1, 5, 1, 4, 1, 2, 4, 1, 1, 3, 4, 1]
[1, 5, 3, 1, 5, 5, 2, 5, 4, 2, 1, 1, 1, 4, 4, 5, 2, 1, 1, 5, 5, 5, 4, 5, 3, 4, 4, 4, 5, 1, 5, 2, 5, 4, 4, 5, 1, 4, 2, 5, 5, 2, 4, 4, 1, 2, 1, 4, 5, 4, 1, 1, 3, 2, 3, 1, 4, 2, 1, 4, 1, 2, 1, 5, 3, 1, 3, 5, 3, 1, 4, 1, 1, 5, 5, 3, 4, 3, 1, 1, 5, 1, 1, 5, 4, 1, 5, 5, 5, 5, 4, 2, 4, 3, 3, 4, 4, 3, 5, 1]


In [292]:
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

0.16706947825559204

In [ ]:
K = 3
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 4
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 5
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 6
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 7
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 8
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')

In [ ]:
K = 9
p = predict(mat_test, verbose=False)
f1_score(y_test[:SAMPLE_SIZE], p, average='macro')